In [ ]:
import json
import openai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Set your OpenAI API key
openai.api_key = 'YOUR_API_KEY'

# Example job-specific documents (replace with actual docs)
job_docs = [
    "Python scripting, automation, data analysis using Pandas, NumPy, Matplotlib.",
    "Experience with machine learning frameworks like scikit-learn, TensorFlow, Keras.",
    "Object-oriented programming, version control, API interaction."
]

# Questions to ask
questions = [
    "Can you tell me about your experience with Python?",
    "Describe your experience with machine learning.",
    "How do you approach debugging complex software issues?"
]

# Answers provided by the candidate
candidate_answers = [
    """My experience with Python is quite extensive, spanning several years across various domains. I've primarily leveraged Python for its versatility in data analysis, machine learning, and automation.
    ...""",
    # Add other answers as needed
]

# Function to get embeddings for text
def get_embedding(text):
    response = openai.Embedding.create(input=text, model='text-embedding-ada-002')
    return response['data'][0]['embedding']

# Precompute embeddings for job docs
doc_embeddings = [get_embedding(doc) for doc in job_docs]

# Function to perform RAG retrieval
def rag_retrieve(query, top_k=3):
    query_embedding = get_embedding(query)
    similarities = [cosine_similarity([query_embedding], [doc_emb])[0][0] for doc_emb in doc_embeddings]
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    retrieved_docs = [job_docs[i] for i in top_indices]
    return ' '.join(retrieved_docs)

# Function to evaluate answer
def evaluate_answer(question, answer):
    prompt = f"""You are a professional technical interviewer. 
You will be given a question and a candidate's answer.
Evaluate the answer and provide a short comment on its quality.
Then score it from 1 (poor) to 10 (excellent).

Question: {question}
Answer: {answer}
"""
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    evaluation = response['choices'][0]['message']['content']
    return evaluation

# Loop through questions, perform RAG, evaluate, and save results
results = []

for i, question in enumerate(questions):
    answer = candidate_answers[i]
    # Retrieve relevant docs
    context = rag_retrieve(question)
    # Include context if needed before prompting
    eval_response = evaluate_answer(question, answer)
    # Save results
    result_entry = {
        'question': question,
        'answer': answer,
        'context': context,
        'evaluation': eval_response
    }
    results.append(result_entry)

# Save to JSON file
with open('interview_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Interview evaluation complete. Results saved to 'interview_results.json'.")